In [7]:
#Alur
#Setup
#Buka File
#Lihat data
#Tentukan X dan y
#Cek missing value
#Handle missing value
#Feature Engineering
#Buat Model
#Cari RMSE paling kecil
#fit model ke data test
#Finishing

In [2]:
#Setup
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

print("Setup Done")

Setup Done


In [3]:
#Import Data
data = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/train.csv")
tdata = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/test.csv")
print(data.shape)

(630000, 13)


In [ ]:
#Lihat data
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [14]:
mis = data.isnull().sum()
mis[0:13]

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64

In [11]:
#cari kolom kategorikal untuk model catboost

for col in data:
    if data[col].dtype == "object":
        print(col)

cat_col= [
    'sleep_quality',
    'study_method',
    'facility_rating',
    'internet_access',
    'exam_difficulty'
]

gender
course
internet_access
sleep_quality
study_method
facility_rating
exam_difficulty


In [12]:
#tentukan X dan y
y = data["exam_score"]
X = data.drop(["id","exam_score","gender","course"], axis = 1)
# X = data.drop(["id","exam_score","internet_access","gender","exam_difficulty"], axis = 1)

In [18]:
#factorize category type columns
for col in X.select_dtypes("object"):
    X[col], _ = X[col].factorize()

fcol = X.dtypes ==int

In [19]:
#mi score
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X,y, cat_col):
    mi_scores = mutual_info_regression(X, y, discrete_features= fcol)
    mi_scores = pd.Series(mi_scores, name= "MI Scores", index = X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X,y,fcol)
mi_scores[::3]

study_hours     0.911854
study_method    0.042229
age             0.013022
Name: MI Scores, dtype: float64

In [13]:
#buat model
model = CatBoostRegressor(
    iterations=1500, 
    learning_rate= 0.05,
    depth = 6,
    loss_function= 'RMSE',
    cat_features= cat_col,
    random_seed= 2)

In [14]:
#Train model
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size= 0.2, random_state=1)
model.fit(train_X,train_y)

0:	learn: 18.3129860	total: 374ms	remaining: 9m 20s
1:	learn: 17.7552058	total: 634ms	remaining: 7m 54s
2:	learn: 17.2337352	total: 888ms	remaining: 7m 22s
3:	learn: 16.7252469	total: 1.09s	remaining: 6m 47s
4:	learn: 16.2465379	total: 1.24s	remaining: 6m 11s
5:	learn: 15.7997363	total: 1.42s	remaining: 5m 52s
6:	learn: 15.3848309	total: 1.58s	remaining: 5m 36s
7:	learn: 14.9945224	total: 1.74s	remaining: 5m 24s
8:	learn: 14.6318461	total: 1.88s	remaining: 5m 12s
9:	learn: 14.2907061	total: 2.07s	remaining: 5m 8s
10:	learn: 13.9726175	total: 2.22s	remaining: 5m
11:	learn: 13.6811969	total: 2.39s	remaining: 4m 56s
12:	learn: 13.4049017	total: 2.55s	remaining: 4m 52s
13:	learn: 13.1500944	total: 2.69s	remaining: 4m 45s
14:	learn: 12.8960889	total: 2.88s	remaining: 4m 45s
15:	learn: 12.6560863	total: 3.06s	remaining: 4m 44s
16:	learn: 12.4348525	total: 3.2s	remaining: 4m 39s
17:	learn: 12.2313435	total: 3.33s	remaining: 4m 34s
18:	learn: 12.0402376	total: 3.5s	remaining: 4m 32s
19:	learn:

In [15]:
#predict
preds_vy= model.predict(val_X) #predict jawaban ujian
preds_ty= model.predict(train_X) #predict jawaban latihan soal

In [16]:
#Model Validation
train_rmse = root_mean_squared_error(train_y, preds_ty)
val_rmse = root_mean_squared_error(val_y,preds_vy)
print(train_rmse)
print(val_rmse)

8.73302785876941
8.773170534050665


In [76]:
penting = model.get_feature_importance(prettified=True)
print(penting)

          Feature Id  Importances
0        study_hours    51.183935
1   class_attendance    19.618945
2      sleep_quality     9.709697
3       study_method     8.267085
4    facility_rating     6.664085
5        sleep_hours     4.311290
6                age     0.099738
7             course     0.087366
8    exam_difficulty     0.024533
9             gender     0.022903
10   internet_access     0.010425


In [82]:
#fit model ke Semua data
model.fit(X,y)

0:	learn: 18.3006156	total: 403ms	remaining: 10m 3s
1:	learn: 17.7249597	total: 684ms	remaining: 8m 32s
2:	learn: 17.1877488	total: 922ms	remaining: 7m 39s
3:	learn: 16.6840429	total: 1.25s	remaining: 7m 47s
4:	learn: 16.2128331	total: 1.49s	remaining: 7m 26s
5:	learn: 15.7689124	total: 1.76s	remaining: 7m 17s
6:	learn: 15.3533279	total: 2.03s	remaining: 7m 13s
7:	learn: 14.9637138	total: 2.31s	remaining: 7m 9s
8:	learn: 14.6000579	total: 2.58s	remaining: 7m 7s
9:	learn: 14.2637350	total: 2.85s	remaining: 7m 5s
10:	learn: 13.9476548	total: 3.1s	remaining: 6m 59s
11:	learn: 13.6400434	total: 3.4s	remaining: 7m 1s
12:	learn: 13.3516101	total: 3.65s	remaining: 6m 58s
13:	learn: 13.0827177	total: 3.89s	remaining: 6m 53s
14:	learn: 12.8345258	total: 4.1s	remaining: 6m 46s
15:	learn: 12.6016736	total: 4.35s	remaining: 6m 43s
16:	learn: 12.3586004	total: 4.61s	remaining: 6m 42s
17:	learn: 12.1328510	total: 4.84s	remaining: 6m 38s
18:	learn: 11.9258125	total: 5.09s	remaining: 6m 36s
19:	learn:

In [93]:
#Prediksi test data
tdata_fit= tdata.drop("id", axis=1)
pred_test= model.predict(tdata_fit)

In [89]:
print(X.columns)
print(tdata.columns)

Index(['age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty'],
      dtype='object')
Index(['age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty'],
      dtype='object')


In [95]:
#Buat ke dataset
submit = pd.DataFrame({"id":tdata.id, "exam_score": pred_test})
#Buat dataset ke file
submit.to_csv("P1.csv", index= False)